# Testing Agents in AI Foundry
This notebook creates and runs an agent from AI Foundry

In [5]:
# Install required packages

!pip install requests "semantic-kernel[azure]>=1.34.0"

## Step 2
Create Project client

In [12]:
import asyncio
from datetime import date
import logging
import os
from azure.identity import DefaultAzureCredential, AzureDeveloperCliCredential

from semantic_kernel.agents import (
    AzureAIAgent,
    AzureAIAgentThread,
    AzureAIAgentSettings,
)

from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv(override=True)

endpoint = os.environ.get("AZURE_AI_FOUNDRY_CONNECTION_STRING")
deployment_name = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", None)
tenant_id = os.environ.get("AZURE_TENANT_ID", None)

ai_agent_settings = AzureAIAgentSettings(
    endpoint=endpoint,
    model_deployment_name=deployment_name,
    api_version=api_version,
)
print(ai_agent_settings)

creds = AzureDeveloperCliCredential(tenant_id=tenant_id) if os.environ.get("USE_AZURE_DEV_CLI") == "true" else DefaultAzureCredential()
client = AzureAIAgent.create_client(
    credential=creds,
    endpoint=ai_agent_settings.endpoint,
    api_version=ai_agent_settings.api_version,
)

agent_definition = None
agent_name = "Joe"

# List agents
print("\n --- Agents ---")
async for agent in client.agents.list_agents():
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Description: {agent.description}, Deployment Name: {agent.model}")
    if agent.name == agent_name and agent_definition is None:
        agent_definition = agent

print("\n --- Connections ---")
# List connections
async for connection in client.connections.list():
    print(f"Connection ID: {connection.id}, Name: {connection.name}, Type: {connection.type} Default: {connection.is_default}")

env_file_path=None env_file_encoding='utf-8' model_deployment_name='gpt-4.1' endpoint='https://ai-foundry-no-models-b7hwurvm66omu.services.ai.azure.com/api/projects/ai-project--my-app-1' agent_id=None bing_connection_id=None azure_ai_search_connection_id=None azure_ai_search_index_name=None api_version=None

 --- Agents ---
Agent ID: asst_LbOJceIvFhCy3J4rnmse5FcV, Name: Joe, Description: None, Deployment Name: gpt-35-turbo
Agent ID: asst_crrmBGZW1bUJJCVnKyKmyHgm, Name: Joe, Description: None, Deployment Name: gpt-35-turbo
Agent ID: asst_2NTwCuZAZujo976pDk68gXEk, Name: Agent138, Description: None, Deployment Name: gpt-35-turbo

 --- Connections ---
Connection ID: /subscriptions/ece240d5-5c85-4dba-8829-29b9949adad1/resourceGroups/my-app-1-rg/providers/Microsoft.CognitiveServices/accounts/ai-foundry-no-models-b7hwurvm66omu/projects/ai-project--my-app-1/connections/applicationInsights, Name: applicationInsights, Type: ConnectionType.APPLICATION_INSIGHTS Default: True
Connection ID: /subscr

### Create Agent

In [ ]:

agent_instructions = (
    "You are a helpful assistant named Joe. "
    "You are designed to assist with various tasks and answer questions. "
    "Your responses should be concise and informative."
)

if agent_definition:
    print(f"Found existing agent with ID: {agent_definition.id} and name: {agent_definition.name}")
    agent_definition = await client.agents.update_agent(
        agent_id=agent_definition.id,
        instructions=agent_instructions,
        model=ai_agent_settings.model_deployment_name,
    )
else:
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=agent_name,
        instructions=agent_instructions,
    )
    print(f"Created agent with id {agent_definition.id} name: {agent_name} with model {ai_agent_settings.model_deployment_name}")

agent = AzureAIAgent(
    client=client,
    definition=agent_definition,
)

Found existing agent with ID: asst_LbOJceIvFhCy3J4rnmse5FcV and name: Joe
Created agent with id asst_LbOJceIvFhCy3J4rnmse5FcV name: Joe with model gpt-4.1


### Use the agent

In [14]:
input = "Tell me a joke about Cloud Computing - AWS, GCP and Azure go into a bar."
thread = AzureAIAgentThread(client=client)

async for agent_response in agent.invoke(messages=input, thread=thread):
    print(f"Agent: {agent_response}")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fb884482590>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fb87f4c4e50>


AgentInvokeException: Run failed with status: `RunStatus.FAILED` for agent `Joe` and thread `thread_LRaVUhE0UrFSfhjFtxWlhkAt` with error: Failed to resolve model info for: gpt-4.1